# Aula 01 - Mineração de Textos

Professor: Luiz Frias 
- Email: l.frias@poli.ufrj.br
- Linkedin: in/luizfrias/
- Twitter: @lfdefrias

#### O que? 
Mineração de Textos é o processo de extração de informação de fontes de dados escritas com o uso de um computador.

#### Para que? 

São muitas as bases textuais relevantes: notícias jornalísticas e seu impacto na sociedade, depoimentos de consumidores de marcas em mídias sociais, entre outras. 

#### Como? 

Algumas fontes de dados disponibizam os dados através de Interface de Programação de Aplicações (API). As APIs representam a forma mais cômoda de extrair os dados e devem ser utilizadas sempre que possível.

Há casos em que isso não ocorre, como nos portais de notícias. Assim, a maneira de extrair o conteúdo das notícias publicadas é tipicamente através de uma técnica conhecida como web scraping, onde a estrutura da página web (DOM) é explorada com a finalidade de extrair o texto.

Há ainda casos onde os documentos textuais foram digitalizados e são disponibilizados como imagens, situação comum em livros, contratos, e outros documentos. Quando isso ocorre, uma técnica a ser explorada é o Reconhecimento Ótico de Caracteres (OCR).

Abaixo vamos estudar como utilizar uma API, um exemplo de web scraping e como extrair dados de uma imagem com o uso de  OCR.

### API 

Exemplos do uso de API para recuperar informações textuais.

Vamos usar o módulo `requests` para recuperar os dados das APIs e o módulo `pandas` para explorar seus dados. Para instalar: `$ conda install requests pandas`.

In [2]:
import requests
import pandas as pd

#### Câmara dos Deputados 

API: https://dadosabertos.camara.leg.br/swagger/api.html

Recomendo que entrem na página e leiam a documentação.

1. Exemplo de uma requisição

Vamos começar utilizando o endpoint `/deputados` e analisando a resposta.

In [3]:
response = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados")
response.text[:100]

'{"dados":[{"id":204554,"uri":"https://dadosabertos.camara.leg.br/api/v2/deputados/204554","nome":"Ab'

In [4]:
response.headers['Content-Type']

'application/json;charset=UTF-8'

2. Alterando o tipo de resposta

Como vimos no exemplo acima, o formato da resposta foi um JSON (https://www.json.org/json-en.html), embora nada tenha sido especificado. Em Python essa costuma ser a escolha pela facilidade de manipulação da estrutura de dados (um JSON é interpretado como um dicionário). Mas em outras situações pode ser mais interessante a manipulação de XML.

Cada API possui uma forma de mudar o formato de resposta. Em algumas espera-se que ele seja passado na URL (por ex.: https://dadosabertos.camara.leg.br/api/v2/deputados.json e https://dadosabertos.camara.leg.br/api/v2/deputados.xml). Em outras, espera-se que seja feito o uso do header HTTP de Content-Type (application/json;charset=UTF-8 e application/xml.charset=UTF-8). 

Já nesta API, espera-se que um header `accept` seja configurado. Vamos alterar a resposta para trabalhar com um XML.

In [5]:
headers = {'accept': 'application/xml'}
response = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados", headers=headers)
response.text[:100]

'<xml><dados><deputado_><id>204554</id><uri>https://dadosabertos.camara.leg.br/api/v2/deputados/20455'

In [7]:
response.headers['Content-Type']

'application/xml;charset=UTF-8'

3. Explorando o resultado

Vamos voltar ao formato JSON original. Desta vez especificando que deve ser retornado um JSON para evitar surpresas.

In [8]:
headers = {'accept': 'application/json'}
response = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados", headers=headers)

Podemos manipular esta resposta de duas formas:
    
    - Passando a variável `response.text` para um parser de json, obtendo assim um dicionário
    - Ou utilizando o método `response.json()`, que faz isso automaticamente por nós.

In [12]:
response.json().keys()

dict_keys(['dados', 'links'])

A seção `links` é útil para paginação dos resultados, possível através da manipulação de parâmetros da API. 

In [13]:
response.json()['links']

[{'rel': 'self',
  'href': 'https://dadosabertos.camara.leg.br/api/v2/deputados'},
 {'rel': 'first',
  'href': 'https://dadosabertos.camara.leg.br/api/v2/deputados?pagina=1&itens=1000'},
 {'rel': 'last',
  'href': 'https://dadosabertos.camara.leg.br/api/v2/deputados?pagina=1&itens=1000'}]

A seção `dados` retorna uma lista de dicionários, com um registro por deputado. Vamos olhar o primeiro.

In [18]:
print (len(response.json()['dados']), 'deputados')

513 deputados


In [19]:
response.json()['dados'][0]

{'id': 204554,
 'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204554',
 'nome': 'Abílio Santana',
 'siglaPartido': 'PL',
 'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
 'siglaUf': 'BA',
 'idLegislatura': 56,
 'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204554.jpg',
 'email': 'dep.abiliosantana@camara.leg.br'}

Com o auxílio do `pandas` é muito fácil transformar um dicionário em uma tabela para manipular a resposta com mais facilidade. Ao passar uma lista de dicionários no construtor, ele interpreta cada registro da lista como uma linha e cada par usuário/valor do dicionário como uma coluna/valor da célula na tabela criado.

In [22]:
pd.DataFrame(response.json()['dados']).head(5)

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,204554,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,PL,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliosantana@camara.leg.br
1,204521,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.abouanni@camara.leg.br
2,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,PROS,https://dadosabertos.camara.leg.br/api/v2/part...,AP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
3,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br
4,204528,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Ventura,NOVO,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.adrianaventura@camara.leg.br


4. Passando parâmetros

Na documentação é possível ver uma lista de parâmetros aceitos pelo endpoint. Vamos explorar seu uso.

In [36]:
params = {'nome': 'maia'} # define-se um novo dicionario com os parametros a serem passados

headers = {'accept': 'application/json'}
response = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados", headers=headers
                        , params=params) # usa-se esse novo dicionario como argumento da funcao
pd.DataFrame(response.json()['dados']).head(5)

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,4237,https://dadosabertos.camara.leg.br/api/v2/depu...,ALCIDES MAIA,None,None,RS,32,https://www.camara.leg.br/internet/deputado/ba...,None
1,136226,https://dadosabertos.camara.leg.br/api/v2/depu...,ALEXANDRE MAIA,PMDB,https://dadosabertos.camara.leg.br/api/v2/part...,MG,52,https://www.camara.leg.br/internet/deputado/ba...,None
2,4166,https://dadosabertos.camara.leg.br/api/v2/depu...,ALFREDO DE MAIA,None,None,AL,31,https://www.camara.leg.br/internet/deputado/ba...,None
3,130300,https://dadosabertos.camara.leg.br/api/v2/depu...,ÁLVARO MAIA,None,None,AM,36,https://www.camara.leg.br/internet/deputado/ba...,None
4,130526,https://dadosabertos.camara.leg.br/api/v2/depu...,ANTÔNIO MAIA,PSD,https://dadosabertos.camara.leg.br/api/v2/part...,AM,40,https://www.camara.leg.br/internet/deputado/ba...,None


5. Recuperando pronunciamentos

Vamos trabalhar com texto agora (finalmente!). Para isso, vamos utilizar o endpoint `/deputados/{id}/discursos` e recuperar o último pronunciamento do deputado Rodrigo Maia.

Como o endpoint espera o id do deputado, primeiro vamos descobrir qual é esse identificador.

In [37]:
params = {'nome': 'rodrigo maia'} 

headers = {'accept': 'application/json'}
response = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados", headers=headers
                        , params=params) 
pd.DataFrame(response.json()['dados']).head(5)

,id,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,74693,https://dadosabertos.camara.leg.br/api/v2/depu...,Rodrigo Maia,DEM,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,56,https://www.camara.leg.br/internet/deputado/ba...,dep.rodrigomaia@camara.leg.br


De posse do identificador do deputado, vamos estabelecer os parâmetros necessários para trazer apenas o último discurso.

In [28]:
params = {
    'ordenarPor': 'dataHoraInicio', # ordenar por data de início do pronunciamento
    'ordem': 'desc', # último discurso primeiro
    'itens': 1, # trazer apenas o primeiro da lista
    'dataInicio': '2020-01-01' # por padrão o endpoint trabalha apenas com a última semana de dados, vamos ampliar a busca
} 

headers = {'accept': 'application/json;utf-8'}
response = requests.get(
    "https://dadosabertos.camara.leg.br/api/v2/deputados/74693/discursos", # o id do deputado é passado na URL
    headers=headers,
    params=params) 

Vamos analisar a resposta da API

In [29]:
response.json()['dados']

[{'dataHoraInicio': '2020-08-26T11:52',
  'dataHoraFim': None,
  'uriEvento': '',
  'faseEvento': {'titulo': 'Homenagem',
   'dataHoraInicio': None,
   'dataHoraFim': None},
  'tipoDiscurso': 'HOMENAGEM',
  'urlTexto': 'http://imagem.camara.leg.br/dc_20b.asp?largura=&altura=&tipoForm=diarios&selCodColecaoCsv=J&Datain=27%2F8%2F2020&txPagina=&txSuplemento=&enviar=Pesquisar',
  'urlAudio': None,
  'urlVideo': None,
  'keywords': 'SESSÃO CONJUNTA, HOMENAGEM, PROMULGAÇÃO, PEC 15/2015, PROPOSTA DE EMENDA À CONSTITUIÇÃO, FUNDO DE MANUTENÇÃO E DESENVOLVIMENTO DA EDUCAÇÃO BÁSICA E DE VALORIZAÇÃO DOS PROFISSIONAIS DA EDUCAÇÃO (FUNDEB), CARÁTER PERMANENTE.',
  'sumario': 'Saudações aos Parlamentares presentes e aos participantes na sessão virtual. Importância da promulgação da Emenda Constitucional nº 108, de 2020, sobre a transformação do Fundo de Manutenção e Desenvolvimento da Educação Básica e de Valorização dos Profissionais da Educação - FUNDEB, em instrumento permanente de financiamento da

O pronunciamento, finalmente.

In [30]:
print ( response.json()['dados'][0]['transcricao'] )

O SR. RODRIGO MAIA (DEM - RJ. Para discursar. Sem revisão do orador.) - Bom dia a todos.
Cumprimento o Presidente Davi Alcolumbre, a Deputada Professora Dorinha Seabra Rezende e, por meio da Deputada, todas as minhas colegas Deputadas presentes e as que estão acompanhando a sessão virtual; a Deputada Raquel Muniz, autora da PEC; o Senador Eduardo Braga e o Deputado Ricardo Barros, Líderes do Governo; o Senador Weverton; as Sras. Deputadas e os Srs. Deputados, as Sras. Senadoras e os Srs. Senadores, Congressistas.
Hoje, certamente, é um dia muito especial para o Congresso Nacional, mas principalmente para milhões de brasileiros, que passam a ter, na Constituição Federal, a garantia da priorização da educação pública no País. 
Todos nós que participamos, ou pelo menos acompanhamos - boa parte de nós -, do início da execução do FUNDEF, que depois virou FUNDEB, e agora é o FUNDEB para a educação básica permanente, sabemos da importância que esse programa teve e certamente terá nos próximos

### Web Scraping 

Exemplos de como extrair dados a partir de uma página da web.

Vamos usar o módulo `news-please`, implementação de um web crawler genérico. Instale com `$ pip install news-please`.

In [31]:
url = 'https://economia.uol.com.br/todos-a-bordo/2020/08/16/como-pilotos-e-avioes-se-preparam-para-voltar-a-voar-apos-um-tempo-parados.htm'

from newsplease import NewsPlease
article = NewsPlease.from_url(url)

In [32]:
article.title

'Pilotos e aviões têm regras especiais para voltar a voar após a pandemia'

In [33]:
article.authors

['Alexandre Saconi', 'Colaboração Para O Uol', 'Em São Paulo']

In [54]:
print(article.maintext)

A pandemia do novo coronavírus deixou aviões e pilotos sem voar. Quais são os cuidados que os equipamentos e os profissionais exigem durante esse afastamento? Como tem sido o preparo para que todos voltem ao ar em segurança quando houver uma retomada?
Um carro não precisa seguir normas rígidas para ficar parado, e o motorista não necessita passar por novo treinamento após ficar meses sem dirigir, mas na aviação é diferente. Os aviões têm de passar por procedimentos especiais durante o tempo em que estão parados (como fechar todos os orifícios, para não entrar insetos ou água) e no seu retorno. Os pilotos têm de treinar de tempos em tempos.
Pilotos precisam ter voado nos últimos 90 dias
Normas da Anac (Agência Nacional de Aviação Civil) definem que o piloto comercial deve ter experiência recente para poder voar. O piloto precisa ter feito pousos e decolagens em até 90 dias antes do retorno ao trabalho. Tem de ser no mesmo tipo de avião em que vai trabalhar. Isso ficou inviável em divers

### PDF 

Outra aplicação bastante comum é a extração de texto a partir de arquivos PDF.

Uma possibilidade é utilizar módulos que explorem a estrutura deste formato e retornem textos. Um deles é o `pdfminer`, instalado pelo comando `$ pip install pdfminer.six`.

In [56]:
from pdfminer.high_level import extract_text

text = extract_text('in/The Book of Ramen-p8.pdf')
print (text)

 

8 

Introduction 

 

 
 
Thank you for reading this book! This has been a long time coming. Over the last several years I 
have tinkered with the idea of making a book, to help everyone-  from hardcore ramen-lovers to 
those simply intrigued by interesting food- find an all-inclusive resource of recipes and ideas on 
how to make ramen. I felt like it would make sense to make this as widely accessible as 
possible. An e-book format made the most sense, free for anyone to view at any time.  
 
Many of you probably have never had ramen beyond the dry noodle packages found regularly 
at your local grocery store. This book is not about that kind of ramen, which is more akin to 
instant noodles. While tasty in its own right, instant noodles aren’t quite the same thing as the 
dish discussed in this book. Ramen, a dish originating from Japan, is a complex soup that is 
challenging to make and, even to this day, still has quite a bit of mystery. I’m hoping I can at 
least break apart the a

No entanto, nem todo PDF deriva de um documento digitado. Quando o PDF é resultado de um scanner, o resultado é falho, como pode-se ver abaixo.

In [57]:
from pdfminer.high_level import extract_text

text = extract_text('in/400g-p36.pdf')
print (text)

(cid:3)(cid:2)(cid:8)

(cid:4)(cid:1)(cid:1)(cid:7)(cid:8)

(cid:18)(cid:25)

(cid:12)(cid:25)
(cid:20)(cid:25)

(cid:17)(cid:25)
(cid:23)(cid:25)
(cid:7)(cid:8)(cid:25)

(cid:13)(cid:25)
(cid:9)(cid:25)

(cid:2)(cid:25)
(cid:19)(cid:25)
(cid:3)(cid:4)(cid:25)

(cid:21)(cid:25)

(cid:5)(cid:6)(cid:25)
(cid:24)(cid:16)(cid:25)

(cid:15)(cid:25)

(cid:1)(cid:25)

(cid:22)(cid:25)
(cid:5)(cid:8)
(cid:10)(cid:11)(cid:25)
(cid:14)(cid:25)

(cid:27)(cid:49)(cid:69)(cid:46)(cid:69)(cid:93)(cid:89)(cid:38)(cid:103)(cid:152) (cid:83)(cid:40)(cid:90)(cid:116)(cid:52)(cid:70)(cid:62)(cid:40)(cid:152) (cid:62)(cid:53)(cid:76)(cid:40)(cid:49)(cid:38)(cid:152) (cid:38)(cid:152) (cid:125)(cid:83)(cid:152)

(cid:29)(cid:40)(cid:105)(cid:40)(cid:83)(cid:52)(cid:76)(cid:69)(cid:134)(cid:38)(cid:144)(cid:143)(cid:93)(cid:152) (cid:49)(cid:52)(cid:152) (cid:125)(cid:83)(cid:152) (cid:69)(cid:89)(cid:62)(cid:103)(cid:52)(cid:49)(cid:69)(cid:52)(cid:90)(cid:116)(cid:53)(cid:152)

(cid:97)(cid:104)(cid:54)(c

Nesses casos, a melhor alternativa é recorrer a módulos de OCR (reconhecimento ótico de caracteres). Para este exemploo, vamos utilizar o `pytesseract`. As instruções de instalação podem ser encontradas em https://github.com/madmaze/pytesseract#installation

In [59]:
import pytesseract

print(pytesseract.image_to_string('in/400g-p36.png'))

ik

inha

coz

té

4009

1 termos e técnicas de cozinha |

Adicionar manteiga gelada a um
Preparo quente batendo com um fouet
para emulsionar. Isso vai espessd-lo, dar
brilho e textura aveludada. O preparo
nao pode ferver.

Esse termo se refere & consisténcia
de um molho. Quando este recobre as
costas de uma colher com uma camada
fina e translicida (molho velouté,
creme inglés).

Corte transversal e largo especifico
para vegetais de formato cilindrico,

LEAGINO

Frutos, sementes e leguminosas ricos
em gorduras (améndoa, amendoim,
avela, castanha de caju, castanha-do-para,
nozes etc.).

Caramelizagdo de um ingrediente
(normalmente o tomate) em gordura.
Esse processo reduz a acidez, a docura
Ou O amargor excessivo que pode afetar
o resultado final de um molho.

POMI
Ponto em que a manteiga fica em

consisténcia de pomada (quando

deixada em temperatura ambiente).

Formato dado a pastas, mousselines
ou recheios com o uso de duas colhe-
res umidas. No processo, 0 recheio
é "passado" de uma

### Projeto 

Vamos agora montar uma base de dados para utilizarmos no restante do curso.

In [3]:
params = {
    'ordenarPor': 'dataHoraInicio', # ordenar por data de início do pronunciamento
    'ordem': 'desc', # último discurso primeiro
    'itens': 1000, # trazer apenas o primeiro da lista
    'dataInicio': '2019-01-01' # por padrão o endpoint trabalha apenas com a última semana de dados, vamos ampliar a busca
} 

headers = {'accept': 'application/json;utf-8'}
response = requests.get(
    "https://dadosabertos.camara.leg.br/api/v2/deputados/74693/discursos", # o id do deputado é passado na URL
    headers=headers,
    params=params
) 

In [6]:
dados = response.json()['dados']

In [8]:
print ('%d discursos' % len(dados))

31 discursos


In [15]:
pd.DataFrame(dados)[
    ['dataHoraInicio', 'tipoDiscurso', 'keywords', 'sumario', 'transcricao']
].to_csv('out/discursos-maia.csv', sep='|', index=False)

In [19]:
discursos_df = pd.read_csv('out/discursos-maia.csv', sep='|')
discursos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dataHoraInicio  31 non-null     object
 1   tipoDiscurso    31 non-null     object
 2   keywords        31 non-null     object
 3   sumario         31 non-null     object
 4   transcricao     31 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB


In [21]:
discursos_df.dataHoraInicio = pd.to_datetime(discursos_df.dataHoraInicio)

In [22]:
discursos_df.head(2)

,dataHoraInicio,tipoDiscurso,keywords,sumario,transcricao
0,2020-08-26 11:52:00,HOMENAGEM,"SESSÃO CONJUNTA, HOMENAGEM, PROMULGAÇÃO, PEC 1...",Saudações aos Parlamentares presentes e aos pa...,O SR. RODRIGO MAIA (DEM - RJ. Para discursar. ...
1,2020-08-20 18:40:00,ORIENTAÇÃO DE BANCADA,"MANUTENÇÃO, VETO 17/2020, VETO PARCIAL, PLP 3...",Importância de preservação do teto de gastos p...,O SR. RODRIGO MAIA (DEM - RJ. Para orientar a ...


In [23]:
discursos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   dataHoraInicio  31 non-null     datetime64[ns]
 1   tipoDiscurso    31 non-null     object        
 2   keywords        31 non-null     object        
 3   sumario         31 non-null     object        
 4   transcricao     31 non-null     object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.3+ KB
